In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [ ]:
model_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150,150,3))
model_base.summary()

In [ ]:
base_dir = '../data/cat_dog_full'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

datagen = ImageDataGenerator(rescale=1/255)
batch_size = 20

def extract_feature(directory, sample_count):
    features = np.zeros(shape=(sample_count,4,4,512))
    labels = np.zeros(shape=(sample_count,))
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary'
    )
    
    i = 0
    
    for x_data_batch, t_data_batch in generator:
        feature_batch = model_base.predict(x_data_batch)
        features[i*batch_size:(i+1)*batch_size] = feature_batch
        labels[i*batch_size:(i+1)*batch_size] = t_data_batch
        i += 1
        
        if i*batch_size >= sample_count:
            break
            
    return features, labels

In [ ]:
train_features, train_labels = extract_feature(train_dir, 14000)
validation_features, validation_labels = extract_feature(validation_dir, 6000)

- Classifier

In [ ]:
train_features = np.reshape(train_features, (14000, 4*4*512))
validation_features = np.reshape(validation_features, (6000, 4*4*512))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop

model = Sequential()

model.add(Dense(256, activation='relu',
                input_shape=(4*4*512,)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=RMSprop(learning_rate=2e-5), loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_features, train_labels, epochs=30,
                    validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt

train_acc = history.history['accuracy']
val_acc =  history.history['val_accuracy']

train_loss = history.history['loss']
val_loss =  history.history['val_loss']

plt.plot(train_loss, color='b', label='training loss')
plt.plot(val_loss, color='r', label='validation loss')
plt.legend()
plt.show()

- 조금 더 나은 결과를 얻으려면 데이터가 많아져야 할것 같음
- 증식을 포함하여 다시 작성
- pretrained network와 우리 classifier를 합쳐서 모델을 작성

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import VGG16

In [ ]:
base_dir = '../data/cat_dog_full'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_datagen = ImageDataGenerator(
    rescale=1/255,    
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150,150),
    batch_size=20,
    classes=['cats','dogs'],
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150,150),
    batch_size=20,
    classes=['cats','dogs'],
    class_mode='binary'
)

# pretrained network
model_base = VGG16(include_top=False, weights='imagenet', input_shape=(150,150,3))
# model_base의 weight학습을 동결
model_base.trainable=False

model = Sequential()

# pretrained network를 우리의 모델 앞에 추가
model.add(model_base)

model.add(Flatten(input_shape=(4*4*512,)))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

model.compile(optimizer=RMSprop(learning_rate=2e-5), loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, steps_per_epoch=700, epochs=30,
                    validation_data=validation_generator,
                    validation_steps=300)

## Fine Tuning

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import VGG16


base_dir = '../data/cat_dog_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_datagen = ImageDataGenerator(
    rescale=1/255,    
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150,150),
    batch_size=20,
    classes=['cats','dogs'],
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150,150),
    batch_size=20,
    classes=['cats','dogs'],
    class_mode='binary'
)

# pretrained network
model_base = VGG16(include_top=False, weights='imagenet', input_shape=(150,150,3))
# model_base의 weight학습을 동결
model_base.trainable=False

model = Sequential()

# pretrained network를 우리의 모델 앞에 추가
model.add(model_base)

model.add(Flatten(input_shape=(4*4*512,)))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

model.compile(optimizer=RMSprop(learning_rate=2e-5), loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=100, epochs=30,
                    validation_data=validation_generator,
                    validation_steps=50)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/30
100/100

In [2]:
model_base.trainable=True

# 상위 layer 동결해제
for layer in model_base.layers:
    if layer.name in ['block5_conv1','block5_conv2','block5_conv3']:
        layer.trainable=True
    else:
        layer.trainable=False

# 미세조정이므로 learning_rate를 더 작게 설정
model.compile(optimizer=RMSprop(learning_rate=1e-5), loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=100, epochs=30,
                    validation_data=validation_generator,
                    validation_steps=50)

Epoch 1/30
100/100 [==============================] - 14s 137ms/step - loss: 0.4021 - accuracy: 0.8145 - val_loss: 0.2861 - val_accuracy: 0.8840
Epoch 2/30
100/100 [==============================] - 14s 135ms/step - loss: 0.3277 - accuracy: 0.8455 - val_loss: 0.2364 - val_accuracy: 0.8940
Epoch 3/30
100/100 [==============================] - 14s 136ms/step - loss: 0.3162 - accuracy: 0.8625 - val_loss: 0.2276 - val_accuracy: 0.9020
Epoch 4/30
100/100 [==============================] - 14s 135ms/step - loss: 0.2989 - accuracy: 0.8730 - val_loss: 0.2219 - val_accuracy: 0.9040
Epoch 5/30
100/100 [==============================] - 14s 135ms/step - loss: 0.2862 - accuracy: 0.8715 - val_loss: 0.2264 - val_accuracy: 0.8920
Epoch 6/30
100/100 [==============================] - 13s 135ms/step - loss: 0.2769 - accuracy: 0.8750 - val_loss: 0.2092 - val_accuracy: 0.9070
Epoch 7/30
100/100 [==============================] - 14s 135ms/step - loss: 0.2635 - accuracy: 0.8820 - val_loss: 0.2389 - val_ac